<a href="https://colab.research.google.com/github/FatemehNMT/Visual-SLAM-Book-Google-Colab/blob/main/CurveFittingWithGauss_Newton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The Gauss-Newton algorithm is a numerical optimization method used to find the best-fitting parameters for a non-linear function to a set of data points, minimizing the sum of squared residuals. It's a variation of Newton's method specifically tailored for least squares problems, where the goal is to minimize the difference between observed and predicted values.\

https://en.wikipedia.org/wiki

**Problem Setup:**


*   You have a set of data points \((x*i*, y*i*)\) and a non-linear model function \(f(x,\mathbf*p*)\), where \(\mathbf{p}\) represents the parameters you want to optimize.
*   The goal is to find the parameter values \(\mathbf*p*\) that minimize the sum of squared errors (residuals):\



**Iterative Approach:**


*   Initialization
*   Linear Approximation
*   Jacobian Matrix
*   Update Step



**Matrix decomposition**, also known as **matrix factorization**, is a powerful technique used to solve systems of linear equations by breaking down the coefficient matrix into simpler, structured matrices. Common methods include **LU** decomposition, **QR** decomposition, and Singular Value Decomposition (**SVD**).\
**References:**\
https://www.youtube.com/watch?v=m3EojSAgIao&t=7

https://sciendo.com/2/download/p-heiZdOcjkpHfda~ywgf4zkjVhgeYCb8c0VekXGb~.pdf

# **Chapter 5: Nonlinear Optimization**

**Chapter Reference:** https://github.com/gaoxiang12/slambook2/tree/master/ch6

## **Mount**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


## **Install Eigen**


In [ ]:
!git clone https://gitlab.com/libeigen/eigen.git

Cloning into 'eigen'...
remote: Enumerating objects: 125784, done.
remote: Counting objects: 100% (488/488), done.
remote: Compressing objects: 100% (192/192), done.
remote: Total 125784 (delta 319), reused 456 (delta 296), pack-reused 125296 (from 1)
Receiving objects: 100% (125784/125784), 105.57 MiB | 13.27 MiB/s, done.
Resolving deltas: 100% (104195/104195), done.


In [ ]:
%cd eigen

/content/eigen


In [ ]:
!mkdir build

In [ ]:
%cd build

/content/eigen/build


In [ ]:
!cmake ..

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- The Fortran compiler identification is GNU 11.4.0
-- Performing Test standard_math_library_linked_to_automatically
-- Performing Test standard_math_library_linked_to_automatically - Success
-- Standard libraries to link to explicitly: none
-- Performing Test COMPILER_SUPPORT_WERROR
-- Performing Test COMPILER_SUPPORT_WERROR - Success
-- Performing Test COMPILER_SUPPORT_pedantic
-- Performing Test COMPILER_SUPPORT_pedantic - Success
-- Performing Test COMPILER_SUPPORT_Wall
-- Performing T

In [ ]:
!sudo make install

[  0%] Building CXX object blas/CMakeFiles/eigen_blas_static.dir/single.cpp.o
[  0%] Building CXX object blas/CMakeFiles/eigen_blas_static.dir/double.cpp.o
[  0%] Building CXX object blas/CMakeFiles/eigen_blas_static.dir/complex_single.cpp.o
[  0%] Building CXX object blas/CMakeFiles/eigen_blas_static.dir/complex_double.cpp.o
[  0%] Building CXX object blas/CMakeFiles/eigen_blas_static.dir/xerbla.cpp.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/srotm.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/srotmg.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/drotm.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/drotmg.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/lsame.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/dspmv.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/ssbmv.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_s

In [ ]:
%cd /content/

/content


## **Examples**

### **SLAM Book 2: Curve Fitting with Gauss-Newton**

**Page 110**\
https://github.com/gaoxiang12/slambook2/blob/master/ch6/gaussNewton.cpp

In [ ]:
!mkdir MyExample
%cd MyExample

/content/MyExample


In [ ]:
%%writefile CMakeLists.txt

cmake_minimum_required(VERSION 3.33)
project(ch6)

set(CMAKE_BUILD_TYPE Release)
set(CMAKE_CXX_FLAGS "-std=c++14 -O3")

list(APPEND CMAKE_MODULE_PATH ${PROJECT_SOURCE_DIR}/cmake)

# OpenCV
find_package(OpenCV REQUIRED)
include_directories(${OpenCV_INCLUDE_DIRS})

# Eigen
include_directories("/content/eigen")

add_executable(gaussNewton gaussNewton.cpp)
target_link_libraries(gaussNewton ${OpenCV_LIBS})


Writing CMakeLists.txt


In [ ]:
%%writefile gaussNewton.cpp

#include <iostream>
#include <chrono>
#include <opencv2/opencv.hpp>
#include <Eigen/Core>
#include <Eigen/Dense>

using namespace std;
using namespace Eigen;

int main(int argc, char **argv) {
  double ar = 1.0, br = 2.0, cr = 1.0;         // True parameter value
  double ae = 2.0, be = -1.0, ce = 5.0;        // Estimated parameter values
  int N = 100;                                 // Data Points
  double w_sigma = 1.0;                        // Noise Sigma
  double inv_sigma = 1.0 / w_sigma;
  cv::RNG rng;                                 // OpenCV Random Number Generator

  vector<double> x_data, y_data;               // data
  for (int i = 0; i < N; i++) {
    double x = i / 100.0;
    x_data.push_back(x);
    y_data.push_back(exp(ar * x * x + br * x + cr) + rng.gaussian(w_sigma * w_sigma));
  }

  // Start Gauss-Newton iteration
  int iterations = 100;    // Iterations
  double cost = 0, lastCost = 0;  // The cost of this iteration and the cost of the previous iteration

  chrono::steady_clock::time_point t1 = chrono::steady_clock::now();
  for (int iter = 0; iter < iterations; iter++) {

    Matrix3d H = Matrix3d::Zero();             // Hessian = J^T W^{-1} J in Gauss-Newton
    Vector3d b = Vector3d::Zero();             // bias
    cost = 0;

    for (int i = 0; i < N; i++) {
      double xi = x_data[i], yi = y_data[i];  // The i-th data point
      double error = yi - exp(ae * xi * xi + be * xi + ce);
      Vector3d J; // Jacobian matrix
      J[0] = -xi * xi * exp(ae * xi * xi + be * xi + ce);  // de/da
      J[1] = -xi * exp(ae * xi * xi + be * xi + ce);  // de/db
      J[2] = -exp(ae * xi * xi + be * xi + ce);  // de/dc

      H += inv_sigma * inv_sigma * J * J.transpose();
      b += -inv_sigma * inv_sigma * error * J;

      cost += error * error;
    }

    // Solving Linear Equations Hx=b
    Vector3d dx = H.ldlt().solve(b);
    if (isnan(dx[0])) {
      cout << "result is nan!" << endl;
      break;
    }

    if (iter > 0 && cost >= lastCost) {
      cout << "cost: " << cost << ">= last cost: " << lastCost << ", break." << endl;
      break;
    }

    ae += dx[0];
    be += dx[1];
    ce += dx[2];

    lastCost = cost;

    cout << "total cost: " << cost << ", \t\tupdate: " << dx.transpose() <<
         "\t\testimated params: " << ae << "," << be << "," << ce << endl;
  }

  chrono::steady_clock::time_point t2 = chrono::steady_clock::now();
  chrono::duration<double> time_used = chrono::duration_cast<chrono::duration<double>>(t2 - t1);
  cout << "solve time cost = " << time_used.count() << " seconds. " << endl;

  cout << "estimated abc = " << ae << ", " << be << ", " << ce << endl;
  return 0;
}

Writing gaussNewton.cpp


* **We will use matrix decomposition to solve linear equations, such as QR, Cholesky, and other decomposition methods. These methods can usually be found in textbooks, such as the matrix theory, and we will not introduce them.**

* **LDLT-decomposition is a generalization of for symmetric matrices which are not positive definite. As opposed to Cholesky decomposition, which exists only for symmetric positive definite matrices, LDLT-decomposition exists for each symmetric matrix.**


In [ ]:
!mkdir build
%cd build

/content/MyExample/MyExample/build


In [ ]:
!cmake ..

CMake Deprecation Warning at CMakeLists.txt:2 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenCV: /usr (found version "4.5.4")
-- Configuring done (0.5s)
-- Generating done (0.0s)
-- Build files have been written to: /content/MyExample/MyExample/build


In [ ]:
!make gaussNewton

[ 50%] Building CXX object CMakeFiles/gaussNewton.dir/gaussNewton.cpp.o
[100%] Linking CXX executable gaussNewton
[100%] Built target gaussNewton


In [ ]:
! ./gaussNewton

total cost: 3.19575e+06, 		update: 0.0455771  0.078164 -0.985329		estimated params: 2.04558,-0.921836,4.01467
total cost: 376785, 		update:  0.065762  0.224972 -0.962521		estimated params: 2.11134,-0.696864,3.05215
total cost: 35673.6, 		update: -0.0670241   0.617616  -0.907497		estimated params: 2.04432,-0.0792484,2.14465
total cost: 2195.01, 		update: -0.522767   1.19192 -0.756452		estimated params: 1.52155,1.11267,1.3882
total cost: 174.853, 		update: -0.537502  0.909933 -0.386395		estimated params: 0.984045,2.0226,1.00181
total cost: 102.78, 		update: -0.0919666   0.147331 -0.0573675		estimated params: 0.892079,2.16994,0.944438
total cost: 101.937, 		update: -0.00117081  0.00196749 -0.00081055		estimated params: 0.890908,2.1719,0.943628
total cost: 101.937, 		update:   3.4312e-06 -4.28555e-06  1.08348e-06		estimated params: 0.890912,2.1719,0.943629
total cost: 101.937, 		update: -2.01204e-08  2.68928e-08 -7.86602e-09		estimated params: 0.890912,2.1719,0.943629
cost: 101.937>= last 